# EV Population Data — Cleaning

**Purpose:**  
This notebook cleans the raw EV population dataset based on issues identified
during the audit phase and produces a single analysis-ready dataset.


In [11]:
import pandas as pd
import numpy as np

In [12]:
df_raw = pd.read_csv("../data/raw/Electric_Vehicle_Population_Data.csv")
df = df_raw.copy()
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJYGDEE8L,Thurston,Tumwater,WA,98501.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291.0,35.0,124633715,POINT (-122.89165 47.03954),PUGET SOUND ENERGY INC,5.306701e+10
1,5YJXCAE2XJ,Snohomish,Bothell,WA,98021.0,2018,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238.0,1.0,474826075,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10
2,5YJ3E1EBXK,King,Kent,WA,98031.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220.0,47.0,280307233,POINT (-122.17743 47.41185),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
3,7SAYGDEE4T,King,Issaquah,WA,98027.0,2026,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,41.0,280786565,POINT (-122.03439 47.5301),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
4,WAUUPBFF9G,King,Seattle,WA,98103.0,2016,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16.0,43.0,198988891,POINT (-122.35436 47.67596),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10


In [13]:
def clean_colname(c: str) -> str:
    c = c.strip().lower()
    c = c.replace("&", "and")
    c = c.replace("-", "_").replace(" ", "_")
    c = "".join(ch for ch in c if ch.isalnum() or ch == "_")
    c = "_".join([p for p in c.split("_") if p])  # remove double underscores
    return c

df.columns = [clean_colname(c) for c in df.columns]
df.columns

Index(['vin_1_10', 'county', 'city', 'state', 'postal_code', 'model_year',
       'make', 'model', 'electric_vehicle_type',
       'clean_alternative_fuel_vehicle_cafv_eligibility', 'electric_range',
       'legislative_district', 'dol_vehicle_id', 'vehicle_location',
       'electric_utility', '2020_census_tract'],
      dtype='object')

In [14]:
df = df.dropna(how="all")
df = df.dropna(axis=1, how="all")
print("Shape after empty drops:", df.shape)

Shape after empty drops: (270262, 16)


In [22]:
# Columns by meaning (IDs vs ordered numeric codes)
id_cols = ["postal_code", "2020_census_tract"]
int_cols = ["legislative_district"]

for col in id_cols:
    df[col] = df[col].astype("string")

for col in int_cols:
    df[col] = df[col].astype("Int64")

if id_cols:
    print("ID columns dtypes:")
    print(df[id_cols].dtypes)
    print("\nID columns missing values:")
    print(df[id_cols].isna().sum())

if int_cols:
    print("\nInteger code columns dtypes:")
    print(df[int_cols].dtypes)
    print("\nInteger code columns missing values:")
    print(df[int_cols].isna().sum())


ID columns dtypes:
postal_code          string[python]
2020_census_tract    string[python]
dtype: object

ID columns missing values:
postal_code          10
2020_census_tract    10
dtype: int64

Integer code columns dtypes:
legislative_district    Int64
dtype: object

Integer code columns missing values:
legislative_district    649
dtype: int64



Columns are typed based on meaning, not appearance:
- `postal_code`, `2020_census_tract` → identifiers (`string`)
- `legislative_district` → ordered code (`Int64`)

This prevents data corruption (e.g., lost leading zeros) and ensures correct analysis.
Dtypes and missing values are validated after conversion.


In [15]:
df["model_year"] = (
    df["model_year"]                    
      .astype(str)
      .str.extract(r"(\d{4})")     
      .astype(float)             
      .astype("Int64")  
)

# validation
display(df["model_year"].value_counts(dropna=False).sort_index())
display(df[df["model_year"].isna()].head())


model_year
1999        2
2000        8
2002        1
2003        1
2008       20
2010       23
2011      603
2012     1402
2013     3989
2014     3223
2015     4430
2016     5139
2017     8459
2018    14007
2019    10811
2020    12099
2021    20628
2022    29622
2023    59324
2024    49138
2025    35954
2026    11379
Name: count, dtype: Int64

,vin_1_10,county,city,state,postal_code,model_year,make,model,electric_vehicle_type,clean_alternative_fuel_vehicle_cafv_eligibility,electric_range,legislative_district,dol_vehicle_id,vehicle_location,electric_utility,2020_census_tract


In [17]:
df_clean = df.copy()
df_clean.to_csv("../data/cleaned/ev_population_clean.csv", index=False)

## Data Cleaning Summary

- Standardized column names for consistency and safe programmatic access.
- Removed fully empty rows and columns.
- Converted location-related fields to nullable integers (`Int64`) to eliminate floating-point artifacts while preserving true missing values.
- Parsed and validated `model_year` as a four-digit year with nullable integer type.
- Verified missing values and retained them where they reflect genuine data gaps rather than data quality errors.

**Outcome:**  
The dataset is now analysis-ready. No further cleaning is required.  
Subsequent work proceeds in `03_EDA.ipynb`.